# Technical Test for Data Analyst / Data Scientist / Data Engineer job in DataCompany

In this notebook we will solving step by step the DataCompany's technical test for be parto of the data team.

Here we go!

## Import Libraries

In [1]:
import pandas as pd, numpy as np, plotly.express as px, pymysql, pickle, warnings
from utils.models import GetInfo, DBController, Clean, CREATE_DATABASE, CREATE_TWEETS, INSERT_TWEETS, CREATE_USERS, INSERT_USERS, CREATE_VIEW
from utils.config import HOST, USER, PASSWORD

warnings.filterwarnings("ignore")

## Attacking API Twitter

To attack API Twitter we will use `Tweepy`

`Tweepy` is a Python package to attack API Twitter what have a lot of methods to do it easy, in this case we want to know the tweets what The Bridge | Digital Talent Accelerator is quoted to take their metrics and analyze the sentiment of the tweet

In [2]:
id = 1162694149956603904
start_time = "2022-01-01T00:00:00Z"
end_time = "2022-05-22T23:59:59Z"

In [3]:
# tweets = GetInfo().get_client_tweets(id, start_time, end_time)
# tweets.to_csv("data/tweets.csv", index=False)
tweets = pd.read_csv("data/tweets.csv")
tweets.head()

,Message_id,Message,Date,Author_id,Creator's_Username,Replys,Retweets,Likes,Quotes
0,1526946552492789760,Esta es la historia de Isis🤗 que decidió hacer...,2022-05-18,1162694149956603904,TheBridge_Tech,0,0,0,0
1,1526886949839065088,Gracias @victorianoI y equipo por el análisis ...,2022-05-18,1162694149956603904,victorianoi,0,0,0,0
2,1526842834741039105,¿Cómo combatir el paro juvenil🥵? #tipdeldia En...,2022-05-18,1336771626306854912,TheBridge_Tech,1,4,8,0
3,1526479745185198080,Los perfiles tecnológicos más demandados en el...,2022-05-17,114739070,TheBridge_Tech,0,2,1,0
4,1526133158743359488,La 2da presentación-coloquio del #LibroIC en #...,2022-05-16,15993641,TheBridge_Tech,0,4,4,0


In [4]:
tweets["Message"] = tweets["Message"].apply(Clean().clean_emojis)
tweets["Message"] = tweets["Message"].apply(Clean().remove_links)
tweets["Message"] = tweets["Message"].apply(Clean().signs_tweets)
tweets

,Message_id,Message,Date,Author_id,Creator's_Username,Replys,Retweets,Likes,Quotes
0,1526946552492789760,esta es la historia de isis que decidió hacer ...,2022-05-18,1162694149956603904,TheBridge_Tech,0,0,0,0
1,1526886949839065088,gracias victorianoi y equipo por el análisis c...,2022-05-18,1162694149956603904,victorianoi,0,0,0,0
2,1526842834741039105,cómo combatir el paro juvenil #tipdeldia en un...,2022-05-18,1336771626306854912,TheBridge_Tech,1,4,8,0
3,1526479745185198080,los perfiles tecnológicos más demandados en el...,2022-05-17,114739070,TheBridge_Tech,0,2,1,0
4,1526133158743359488,la da presentación-coloquio del #libroic en #m...,2022-05-16,15993641,TheBridge_Tech,0,4,4,0
...,...,...,...,...,...,...,...,...,...
93,1483469787078533123,del periodismo al marketing digital esta es la...,2022-01-18,1162694149956603904,TheBridge_Tech,0,0,0,0
94,1481340966866989063,ciclo de conferencias sobre inteligencia artif...,2022-01-12,41545659,rayadvd,0,1,6,0
95,1480228635294449667,me encantaría saber que les enseñan sobre esta...,2022-01-09,183583012,TheBridge_Tech,0,0,0,0
96,1479787660910768132,cbusquets hola thebridge_tech te da la opción ...,2022-01-08,10045342,cbusquets,0,0,4,0


## Create Database in RDS

RDS is a tool of AWS for deploy and managment of databases in cloud, we're going to create one previously deployed in the AWS console

In [5]:
db = DBController(HOST, PASSWORD, USER)

Here we instance and execute the `SQL` sentences to create the tables and create a view to show all the information of the database

In [6]:
# db.changeSQL(db="-", query=CREATE_DATABASE, create=True)
# db.changeSQL(db="Prueba_Tecnica", query=CREATE_TWEETS)
# db.changeSQL(db="Prueba_Tecnica", query=CREATE_USERS)
# db.changeSQL(db="Prueba_Tecnica", query=CREATE_VIEW)

# # db.changeSQL(db="Prueba_Tecnica", query="""DROP DATABASE Prueba_Tecnica""")

Here we insert the data to our database

In [7]:
for index, row in tweets.iterrows():

    db.changeSQL(db="Prueba_Tecnica", query=INSERT_TWEETS.format(row["Author_id"], row["Message_id"], 
    row["Message"], row["Date"], row["Replys"], row["Retweets"], row["Likes"], row["Quotes"]))
    
    try:
        db.changeSQL(db="Prueba_Tecnica", query=INSERT_USERS.format(row["Author_id"], row["Creator's_Username"]))
    except pymysql.IntegrityError:
        continue

# Exploratory Data Analysis

In this point we are going to discover:
1. `What is the tweet with most impact`
2. `Who is the user who quote the school the most`
3. `What is the month with the most number of tweets`
4. `What are the most repeated words`
5. `What kind of mathematical correlation are between the public metrics`

In [8]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Message_id          98 non-null     int64 
 1   Message             98 non-null     object
 2   Date                98 non-null     object
 3   Author_id           98 non-null     int64 
 4   Creator's_Username  98 non-null     object
 5   Replys              98 non-null     int64 
 6   Retweets            98 non-null     int64 
 7   Likes               98 non-null     int64 
 8   Quotes              98 non-null     int64 
dtypes: int64(6), object(3)
memory usage: 7.0+ KB


Solving the point 1:

Calculating the impact of the tweet

In [9]:
tweets["Impact"] = [reply + retweet + like + quote for reply, retweet, like, quote in
                    zip(tweets["Replys"], tweets["Retweets"], tweets["Likes"], tweets["Quotes"])]

In [10]:
fig = px.histogram(tweets, x="Message_id", y="Impact", nbins=10, color="Message_id", hover_data=["Message_id", "Impact"])
fig.update_layout(bargap=0.2, title="Impact of the tweets")
fig.show()

In the graph we look that the max impact of a tweet is 23, let's got to know what is it

Looking up the tweet

In [11]:
tweets.sort_values(by="Impact", ascending=False).head(1)

,Message_id,Message,Date,Author_id,Creator's_Username,Replys,Retweets,Likes,Quotes,Impact
73,1493902011242274825,muy #agradecida del gran #trabajo en #equipo p...,2022-02-16,2819387949,TheBridge_Tech,1,9,13,0,23


In [12]:
np.array(tweets["Message"][tweets["Message_id"] == "1493902011242274825"]) 

array([], dtype=object)

Solving point 2:

In [13]:
tweets.groupby("Creator's_Username").count().sort_values(by="Message", ascending=False).head(3)

,Message_id,Message,Date,Author_id,Replys,Retweets,Likes,Quotes,Impact
Creator's_Username,,,,,,,,,
TheBridge_Tech,42,42,42,42,42,42,42,42,42
bertinha84,5,5,5,5,5,5,5,5,5
FUniversia,3,3,3,3,3,3,3,3,3


In [14]:
fig = px.histogram(tweets, x="Creator's_Username", nbins=10, color="Creator's_Username", hover_data=["Creator's_Username"])
fig.update_layout(bargap=0.2, title="Users who most quotes to the school")
fig.show()

We see that, by difference, the account that quotes The Bridge the most is itself, this is due to the autoretweets and self-quotes that are usually done to show recent tweets so that they have more impact, followed by this, the account that quotes the most The Bridge different from itself is `bertinha84`

Solving point 3

In [15]:
tweets["Date"] = pd.to_datetime(tweets["Date"])
tweets["Month"] = list(map(lambda x: x.month, tweets["Date"]))
tweets["Month"] = tweets["Month"].map({1 : "January", 2 : "February", 3 : "March", 4 : "April", 5: "May"})

In [16]:
fig = px.histogram(tweets, x="Month", nbins=5, color="Month")
fig.update_layout(bargap=0.2, title="Users who most quotes to the school")
fig.show()

We can see that in the month of April there is the highest concentration of tweets among the months studied, this may be due to the fact that the activity must be greater so that it has more reach before the summer months, which perhaps can be You must want to attract people who want to take advantage of that time that is relatively freer than other times of the year to learn something new in a quick training such as a bootcamp

Solving point 4:

Solving point 5:

In [17]:
correlation = tweets[["Replys", "Retweets", "Likes", "Quotes"]].corr()
transposed_corr = correlation[::-1]
fig = px.imshow(transposed_corr)
fig.update_layout(title="Math correlation between the public metrics")
fig.show()

We can observe that there is a relatively high correlation between likes and retweets, this may be due to the fact that since they are the most common interaction metrics they go hand in hand in user interaction, that is, if a user likes a content and gives a like, he regularly shares it with a retweet so that it is more easily seen on his timeline for his followers

## Predictions

For the predictions we wil use a pretrained model

In [18]:
with open("data/finished_model.model", "rb") as file:
    model = pickle.load(file)

In [39]:
predictions = model.predict(tweets["Message"])
predictions = pd.Series(predictions, index=[i for i in range(0, 98)])
print("Predictions count\n", predictions.value_counts(), "\n")
print("Predictions count average\n", predictions.value_counts()/98)

Predictions count
 0    64
1    34
dtype: int64 

Predictions count average
 0    0.653061
1    0.346939
dtype: float64


1. We can see that most of the predictions are positive, detailing that 0 is for a text with a positive sentiment and 1 for a negative one.

2. The variables that can be more representative for the prediction can be the positive words such as excellent or funny, as well as the negative ones such as bad or offensive.

3. The model can be improved by removing things like:
    - Hashtags
    - Mentions
    - Abbreviations
    - Misspellings
    - Laughs

4. Another facet where machine learning models can be applied is, for example, the virality of a tweet, that is, based on the text, it is possible to calculate the public metrics or the total sum of interactions.